# Timelapse excersise

In this excersise we will use Python and Raspberry Pi camera to record a timelapse and output a gif file

## Capturing an image

Image cupturing will be performed with a simple call to pycamera2 library.

In [ ]:
import time
from picamera2 import Picamera2, Preview  # Fixed import
from IPython.display import Image, display
import os
from datetime import datetime

def initialize_camera():
    """
    Initialize the PiCamera instance.
    PiCamera インスタンスを初期化します。
    """
    picam2 = Picamera2()
    config = picam2.create_preview_configuration()
    picam2.configure(config)
    return picam2


def capture_image(camera, directory, filename):
    """
    Capture an image and save it to the specified directory with the given filename.
    画像を撮影し、指定されたディレクトリに指定されたファイル名で保存します。
    """
    if not os.path.exists(directory):
        os.makedirs(
            directory
        )  # Create the directory if it doesn't exist. ディレクトリが存在しない場合は作成します。

    filepath = os.path.join(directory, filename)
    camera.start_preview(
        Preview.NULL
    )  # Specify preview type. プレビュータイプを指定します。
    camera.start()  # Start the camera. カメラを起動します。
    time.sleep(
        0.1
    )  # Allow the camera to adjust to lighting conditions. カメラが照明条件に適応するために2秒待ちます。
    camera.capture_file(filepath)
    print(f"Image saved to {filepath}")  # 画像が保存されたことを通知します。
    display(Image(filename=filepath))
    # Return filepath
    return filepath


**take_timelapse_image()** function is responsible for taking images and saving them into specific folder 

In [ ]:
def take_timelapse_image(folder_name):
    # Get current time
    current_time = datetime.now().time()
    # Format the time with underscores as spacers
    formatted_time = current_time.strftime("%H_%M_%S")
    print(formatted_time)
    camera = initialize_camera()
    try:
        capture_image(camera, folder_name, f"{formatted_time}_timelapse.jpg")
    finally:
        camera.stop()  # Ensure the camera is properly closed. カメラが適切に閉じられるようにします。
        camera.stop_preview()  # Stop the preview. プレビューを停止します。
        camera.close()

## Setting up a periodic image taking function

Following function takes in the time period for how long the photos should be taken and time space between taking shots

In [ ]:
def calculate_frame_count(length_mins, frame_time_sec):
    frame_time_min = frame_time_sec / 60
    amount_of_frames = length_mins / frame_time_min
    return amount_of_frames

def take_timelapse(length_mins, frame_time_sec, folder_name):
    frame_count = calculate_frame_count(length_mins, frame_time_sec)
    total_duration = frame_time_sec * frame_count
    print(f"The timelapse will take {frame_count} shots over {total_duration/60} minutes")
    for i in range(int(frame_count)):
        take_timelapse_image(folder_name)
        progress = i/frame_count * 100
        print(f"Taking timelapse.... Current progress: {progress}") 
        time.sleep(frame_time_sec)

## Compiling all the taken pictures into a single timelapse

To simplify the timelapse creation process we will use Python [Pillow](https://pillow.readthedocs.io/en/stable/index.html) library

In [ ]:
from PIL import Image as PilImage
import os

def create_timelapse_gif(image_folder, output_gif_path, duration=0.01):
    # Get all image files from the folder
    print("Compiling timelapse GIF")
    images = []
    for filename in sorted(os.listdir(image_folder)):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(image_folder, filename)
            images.append(PilImage.open(img_path))

    # Check if we have images to work with
    if not images:
        print("No valid image files found in the specified directory.")

    # Save images as a GIF
    images[0].save(
        output_gif_path,
        save_all=True,
        append_images=images[1:],
        duration=duration * 1000,  # convert to milliseconds
        loop=0  # loop indefinitely
    )

    print("Timelapse GIF created")

## Displaying the result

In [ ]:
from IPython.display import Image
def display_gif(gif_path):
    """
    Display a GIF in a Jupyter notebook.
    Parameters:
    gif_path (str): The file path to the GIF you want to display.
    """
    return Image(filename=gif_path)

## Putting everything together

In [ ]:
TIMELAPSE_LENGTH_MINS = 10
TIMELAPSE_FRAME_TIME_SEC = 2
TIMELAPSE_FOLDER_NAME = "test2"
OUTPUT_GIF_PATH = f"{TIMELAPSE_FOLDER_NAME}.gif"
OUTPUT_GIF_DURATION = 0.005

take_timelapse(TIMELAPSE_LENGTH_MINS, TIMELAPSE_FRAME_TIME_SEC, TIMELAPSE_FOLDER_NAME)
create_timelapse_gif(TIMELAPSE_FOLDER_NAME, OUTPUT_GIF_PATH, OUTPUT_GIF_DURATION)
display_gif(OUTPUT_GIF_PATH)